In [18]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from sklearn.metrics import accuracy_score,classification_report,mean_squared_error
import math
from deepctr.models import WDL
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from tensorflow import keras


In [19]:
def RecommenderWDL(train, test, sparse_features, 
                   sequence_features, PK_features, target, batch_size, epochs, 
                   sep=" ",embedding_dim=32,dnn_dropout=0.2,dnn_hidden_units=(256,128), 
                   optimizer='SGD', learning_rate=0.01):
    print('optimizer:{0} | embedding_dim:{1} | lr:{2} | dropout:{3} | hidden_units:{4} | epochs:{5} | batch_size:{6}'.format(optimizer,embedding_dim,
                                                                            learning_rate,dnn_dropout,dnn_hidden_units,
                                                                            epochs, batch_size))
    traindata = train.copy()
    testdata = test.copy()
    
    def split(x):
        key_ans = x.split(sep)
        key_ans = [k.strip() for k in key_ans]
        for key in key_ans:
            if key not in key2index:
                # Notice : input value 0 is a special "padding",so we do not use 0 to encode valid feature for sequence input
                key2index[key] = len(key2index) + 1
        return list(map(lambda x: key2index[x], key_ans))  
    
    traindata[target[0]] = pd.to_numeric(traindata[target[0]], downcast='signed')
    
    testdata[target[0]] = pd.to_numeric(testdata[target[0]], downcast='signed')
    
    data = pd.concat([traindata,testdata], ignore_index = True)
    
    # 1.Label Encoding for sparse features,and process sequence features
    for feat in sparse_features:
        lbe = LabelEncoder()
        #data[feat] = lbe.fit_transform(data[feat])  
        data[feat + "_Encoding"] = lbe.fit_transform(data[feat])
    
    fixlen_feature_columns = [SparseFeat(feat + "_Encoding", data[feat + "_Encoding"].nunique(),embedding_dim=embedding_dim)
                            for feat in sparse_features]
    
    
    linear_feature_columns = fixlen_feature_columns
    dnn_feature_columns = fixlen_feature_columns  
    
    for feat in sequence_features:    
        key2index = {}
        templist = list(map(split, data[feat].values))
        length = np.array(list(map(len, templist)))
        max_len = max(length)
        # Notice : padding=`post`
        templist = pad_sequences(templist, maxlen = max_len, padding = 'post', )
        
        use_weighted_sequence = False
        if use_weighted_sequence:
            varlen_feature_columns = [VarLenSparseFeat(SparseFeat(feat + "_Encoding",vocabulary_size=len(key2index) + 1,
                                        embedding_dim = embedding_dim), maxlen = max_len, combiner='mean',weight_name = feat + "_Encoding" + "_weight")]  # Notice : value 0 is for padding for sequence input feature
        else:
            varlen_feature_columns = [VarLenSparseFeat(SparseFeat(feat + "_Encoding",vocabulary_size= len(key2index) + 1,
                                        embedding_dim = embedding_dim), maxlen = max_len, combiner='mean',weight_name = None)]  # Notice : value 0 is for padding for sequence input feature
    
        dnn_feature_columns += varlen_feature_columns
        linear_feature_columns += varlen_feature_columns    
        data[feat + "_Encoding"] = list(templist)  

    # 2. generate input data for model
    encodedfeat = [feat +'_Encoding' for feat in sparse_features + sequence_features]
    encodedfeat = PK_features + target + encodedfeat

    traindata.drop(columns=target,inplace =True)
    trainEncoded = data.merge(traindata,on = PK_features).loc[:,encodedfeat]

    testdata.drop(columns=target,inplace =True)
    testEncoded = data.merge(testdata,on = PK_features).loc[:,encodedfeat]

    model_input = {feat + "_Encoding":trainEncoded[feat + "_Encoding"] for feat in sparse_features} 
    for feat in sequence_features:
        model_input[feat + "_Encoding"] = np.array([d.T for d in trainEncoded[feat + "_Encoding"].values])

    test_model_input = {feat + "_Encoding":testEncoded[feat + "_Encoding"] for feat in sparse_features} 
    for feat in sequence_features:
        test_model_input[feat + "_Encoding"] = np.array([d.T for d in testEncoded[feat + "_Encoding"].values])

    # 3.Define Model,train,predict and evaluate
    WDLModel = WDL(linear_feature_columns,dnn_feature_columns,dnn_hidden_units=dnn_hidden_units, l2_reg_linear=1e-5,
        l2_reg_embedding=1e-5, l2_reg_dnn=0, seed=1024, dnn_dropout=dnn_dropout, dnn_activation='relu',task='regression')
    
    opt = ''
    
    if optimizer == 'Adam':        
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
        
    WDLModel.compile(opt, loss = "mse")
    
    historyModel = WDLModel.fit(model_input, data[target].values, batch_size=batch_size, epochs=epochs, verbose=2, 
                                validation_split=0.1,)

    # 4. make class predictions with the model and evaluation
    predictions = WDLModel.predict(test_model_input)    
    
    maxrating = testEncoded[target].max()[0]
    minrating = testEncoded[target].min()[0]   
    
    predictions_rounded = []
    for x in predictions:
        if x > maxrating:
            predictions_rounded.append(maxrating)
        elif x < minrating:
            predictions_rounded.append(minrating)
        else:
            predictions_rounded.append(round(x[0]))   
    
    MSE = mean_squared_error(testEncoded[target].values, predictions_rounded)
    print("MSE:",MSE)   
    
    return (WDLModel, MSE)

In [20]:
traindata = pd.read_csv("./Movies_train.csv") 

In [21]:
testdata = pd.read_csv("./Movies_test.csv") 

In [22]:
sparse_features = ['NAME', 'USER_MD5', 'MOVIE_ID', 
                   'DOUBAN_SCORE', 'DOUBAN_VOTES']

sequence_features = ['ACTORS', 'DIRECTORS', 'GENRES', 'REGIONS', 
                     'TAGS', 'LANGUAGES', 'MovieNameWord']

PK_features = ['USER_MD5', 'MOVIE_ID']

target = ['RATING']

epochs = 5

batch_size = 32

In [23]:
model = RecommenderWDL(traindata,testdata, sparse_features, sequence_features,PK_features, target, batch_size, epochs, sep="|")

optimizer:SGD | embedding_dim:32 | lr:0.01 | dropout:0.2 | hidden_units:(256, 128) | epochs:5 | batch_size:32
Epoch 1/5


C:\Users\Montr\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


4960/4960 - 191s - loss: 0.9506 - val_loss: 0.8501
Epoch 2/5
4960/4960 - 253s - loss: 0.8362 - val_loss: 0.8389
Epoch 3/5
4960/4960 - 276s - loss: 0.8015 - val_loss: 0.8174
Epoch 4/5
4960/4960 - 292s - loss: 0.7393 - val_loss: 0.8093
Epoch 5/5
4960/4960 - 290s - loss: 0.6232 - val_loss: 0.8842
MSE: 0.96427196


In [20]:
# Save the model in order that we can use it as much as we want without training it everytime
#model.save("WDLMoviesRatingV2")

In [21]:
# Amazon Electronics

In [4]:
electronicstraindata = pd.read_csv("./df_electronics_train.csv") 

In [293]:
electronicstraindata.columns

Index(['item_id', 'user_id', 'rating', 'timestamp', 'model_attr', 'category',
       'brand', 'user_attr'],
      dtype='object')

In [5]:
electronicstestdata = pd.read_csv("./df_electronics_test.csv") 

In [8]:
sparse_features = ['item_id','timestamp', 'user_id','brand', 'user_attr']

sequence_features = ['model_attr', 'category']

PK_features = ['item_id', 'user_id']

target = ['rating']

sep="&"

epochs = 5

batch_size = 32

In [16]:
tuningConfig = {"batch_size":[32,64],
               "epochs":[3,5],
               "embedding_dim":[32,128],
               "dnn_dropout":[0.2,0.5],
               "dnn_hidden_units":[(128,128),(256,128)],
               "optimizer":["Adam","SGD"],
               "learning_rate":[0.0001,0.01]}



In [17]:
bestmodel={}
bestaccuracy=0.0
for optimizer in tuningConfig["optimizer"]:
    for learning_rate in tuningConfig["learning_rate"]:
        for dnn_hidden_units in tuningConfig["dnn_hidden_units"]:
            for dnn_dropout in tuningConfig["dnn_dropout"]:
                for embedding_dim in tuningConfig["embedding_dim"]:
                    for epochs in tuningConfig["epochs"]:
                        for batch_size in tuningConfig["batch_size"]:                            
                            model, accuracy = RecommenderWDL(electronicstraindata,electronicstestdata, sparse_features, 
                                            sequence_features, PK_features, target, batch_size, epochs, sep,
                                            embedding_dim,dnn_dropout, dnn_hidden_units,optimizer,learning_rate)
                            if accuracy >ccuracy:
                                bestaccuracy = accuracy
                                bestmodel = model

print('The best accuracy is {0}'.format(bestaccuracy))
# Save the model in order that we can use it as much as we want without training it everytime
model.save("WDLelectronicsRating")

optimizer:Adam | embedding_dim:32 | lr:0.0001 | dropout:0.2 | hidden_units:(128, 128) | epochs:3 | batch_size:32
Epoch 1/3


C:\Users\Montr\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1017/1017 - 9s - loss: 3.5352 - val_loss: 1.4116
Epoch 2/3
1017/1017 - 9s - loss: 1.2340 - val_loss: 1.4583
Epoch 3/3
1017/1017 - 9s - loss: 0.6183 - val_loss: 1.5756
MSE: 1.6281824
optimizer:Adam | embedding_dim:32 | lr:0.0001 | dropout:0.2 | hidden_units:(128, 128) | epochs:3 | batch_size:64
Epoch 1/3


C:\Users\Montr\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


509/509 - 5s - loss: 5.6619 - val_loss: 1.4345
Epoch 2/3
509/509 - 5s - loss: 1.4559 - val_loss: 1.4586
Epoch 3/3
509/509 - 5s - loss: 0.8762 - val_loss: 1.5046
MSE: 1.5656409
optimizer:Adam | embedding_dim:32 | lr:0.0001 | dropout:0.2 | hidden_units:(128, 128) | epochs:5 | batch_size:32
Epoch 1/5


...

509/509 - 25s - loss: 1.6889 - val_loss: 1.4711
Epoch 2/3
509/509 - 24s - loss: 1.4749 - val_loss: 1.4630
Epoch 3/3
509/509 - 25s - loss: 1.4556 - val_loss: 1.4992
MSE: 1.5117334
optimizer:SGD | embedding_dim:128 | lr:0.01 | dropout:0.5 | hidden_units:(256, 128) | epochs:5 | batch_size:32
Epoch 1/5


C:\Users\Montr\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1017/1017 - 57s - loss: 1.5916 - val_loss: 1.4723
Epoch 2/5
1017/1017 - 60s - loss: 1.4574 - val_loss: 1.4444
Epoch 3/5
1017/1017 - 64s - loss: 1.4408 - val_loss: 1.4356
Epoch 4/5
1017/1017 - 62s - loss: 1.4300 - val_loss: 1.4397
Epoch 5/5
1017/1017 - 67s - loss: 1.4179 - val_loss: 1.4429
MSE: 1.5102944
optimizer:SGD | embedding_dim:128 | lr:0.01 | dropout:0.5 | hidden_units:(256, 128) | epochs:5 | batch_size:64
Epoch 1/5


C:\Users\Montr\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


509/509 - 28s - loss: 1.6934 - val_loss: 1.4692
Epoch 2/5
509/509 - 29s - loss: 1.4757 - val_loss: 1.4661
Epoch 3/5
509/509 - 27s - loss: 1.4557 - val_loss: 1.4560
Epoch 4/5
509/509 - 31s - loss: 1.4456 - val_loss: 1.4742
Epoch 5/5
509/509 - 30s - loss: 1.4374 - val_loss: 1.4440
MSE: 1.5012176
The best accuracy is 1.762674331665039
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: WDLelectronicsRating\assets


In [26]:
electronicsmodel = RecommenderWDL(electronicstraindata,electronicstestdata, sparse_features, sequence_features, PK_features,
                                  target, batch_size, epochs, sep="&")

optimizer:Adam | embedding_dim:32 | lr:0.0001 | dropout:0.5 | hidden_units:(128, 128) | epochs:3 | batch_size:126
Epoch 1/3


C:\Users\Montr\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


259/259 - 3s - loss: 9.1718 - val_loss: 1.4756
Epoch 2/3
259/259 - 3s - loss: 2.0017 - val_loss: 1.4420
Epoch 3/3
259/259 - 3s - loss: 1.7095 - val_loss: 1.4157
MSE: 1.4980075


In [36]:
pred = pd.Series(electronicsmodel)

In [41]:
# cloth

In [25]:
df_modcloth_train=pd.read_csv("./df_modcloth_train.csv") 

In [6]:
df_modcloth_train['rating'] = pd.to_numeric(df_modcloth_train['rating'], downcast='signed')

In [8]:
df_modcloth_train.shape

(14845, 9)

In [26]:
df_modcloth_test = pd.read_csv("./df_modcloth_test.csv") 

In [27]:
sparse_features = ['item_id', 'user_id','timestamp','size','brand', 'user_attr','category']

sequence_features = ['model_attr']

PK_features = ['item_id', 'user_id']

target = ['rating']

epochs = 5

batch_size = 32

In [28]:
modclothmodel = RecommenderWDL(df_modcloth_train,df_modcloth_test, sparse_features, sequence_features, PK_features,
                                  target, batch_size, epochs, sep="&")

optimizer:SGD | embedding_dim:32 | lr:0.01 | dropout:0.2 | hidden_units:(256, 128) | epochs:5 | batch_size:32
Epoch 1/5


C:\Users\Montr\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


418/418 - 2s - loss: 1.2341 - val_loss: 1.0064
Epoch 2/5
418/418 - 2s - loss: 1.0182 - val_loss: 0.9917
Epoch 3/5
418/418 - 2s - loss: 0.9982 - val_loss: 0.9905
Epoch 4/5
418/418 - 2s - loss: 0.9817 - val_loss: 0.9861
Epoch 5/5
418/418 - 2s - loss: 0.9737 - val_loss: 0.9754
MSE: 1.1091056
